In [2]:

from time import time
import numpy as np
from tst.domutil.util import *
from tst.domutil.test import *
from tst.utils_db import *


from copy import copy
from scipy.sparse import *
from scipy.spatial.distance import *


def Dnorm(hc1, hc2, hcboth):
    log1 =  np.log10( hc1 + 1) 
    log2 =  np.log10( hc1 + 1) 
    log3 =  np.log10( hcboth + 1) 
    norm = (log1 + log2) / 2 - log3 
    return norm

reset_database_connection()

# import sys
reload(sys.modules["tst.domutil.util"])
from tst.domutil.util import *

import sys
reload(sys.modules["tst.domutil.test"])
from tst.domutil.test import *

reload(sys.modules["tst.utils_db"])
from tst.utils_db import *



settings.USE_MODELLER
settings.USE_MODELLER


In [3]:
%%time
reset_database_connection()
sDB = seqDB.objects.get(name = 'CATH-S40-nr')
# sDB = seqDB.objects.get(name = 'CATH_S35')
# sDB.sequence_set.count()
seq = sDB.sequence_set.first()
print seq
# seqset_all = sDB.sequence_set.all().prefetch_related('hmmprofile_set').filter(hit4hmm2hsp__bitscore__gte=30)

# seq.HMMprofile
# seq.hmmprofile
# dir(seq)
# seq.hmmprofile_set.count()


102mA00 from <class 'tst.models.seqDB'>
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.46 ms


In [46]:
qset[0].hit1

<hit4hmm2hsp: Query:HMM for 3.10.105.10.14.0  
Target:2hxwA01 from <class 'tst.models.seqDB'>>

In [44]:
qset[1].

{'_hit1_cache': <hit4hmm2hsp: Query:HMM for 3.10.105.10.14.0  
 Target:2hxwA01 from <class 'tst.models.seqDB'>>,
 '_hit2_cache': <hit4hmm2hsp: Query:HMM for 3.40.190.10.273.0  
 Target:2hxwA01 from <class 'tst.models.seqDB'>>,
 '_state': <django.db.models.base.ModelState at 0x7f41bc935c10>,
 'geoavg': 115.472940553188,
 'hit1_id': 5504717L,
 'hit2_id': 5517696L,
 'id': 5407675L,
 'overlap': 113L}

In [31]:
print HMMprofile.objects.values_list('id','cath_node__parent').count()
print HMMprofile.objects.values_list('id','cath_node__parent').distinct().count()

21087
21087


In [32]:
%%time 
hmm2cath = dict(HMMprofile.objects.values_list('id','cath_node__parent'))

DB_hits = hit4hmm2hsp.objects.filter(target__seqDB=sDB)
vals = list(DB_hits.values())
for x in vals:
    x['node_id']=hmm2cath[x['query_id']]
vals = sorted(vals,key = lambda x: [x['target_id'], x['node_id']])
it = itertools.groupby(vals,key = lambda x:x['target_id'])
it = itertools.chain(*[itertools.combinations(y,2) for x,y in it])
# it = (hit4hmm2hspPR(**
#                     pair__hit4hmm2hspDict(x)
#                    ) for x in it)

CPU times: user 4.8 s, sys: 328 ms, total: 5.13 s
Wall time: 7.79 s


In [76]:
MySQL_hitPR(it,bsize = 5E5)

inserting hit4hmm2hspPR for CATH nodes 0 of -1
inserting hit4hmm2hspPR for CATH nodes 500000 of -1
inserting hit4hmm2hspPR for CATH nodes 1000000 of -1
inserting hit4hmm2hspPR for CATH nodes 1500000 of -1
inserting hit4hmm2hspPR for CATH nodes 2000000 of -1
inserting hit4hmm2hspPR for CATH nodes 2500000 of -1
inserting hit4hmm2hspPR for CATH nodes 3000000 of -1
inserting hit4hmm2hspPR for CATH nodes 3500000 of -1
inserting hit4hmm2hspPR for CATH nodes 4000000 of -1

[SUMMARY]:inserting hit4hmm2hspPR for CATH nodes 
	[Task]: finshed  from  
	[Failrate]0 instances of 4440631 failed, ( 0.00% )
	[Duration]:Ended after 1359.6742 sec


In [52]:
%%time
it = itertools.groupby(vals,key = lambda x:x['target_id'])
it = itertools.chain(*[itertools.combinations(y,2) for x,y in it])
c = counter(range(22140124), INF = 1, per = 0.5E6)
buf = []
for obj in it:
#     c.count()
    jdict = pair__hit4hmm2hspDict(obj)
    if jdict:
        c.count()
        if jdict['overlap']/jdict['geoavg'] > 0.5:
#         if 1:
            buf += [jdict]
c.summary()
print len(buf)    
#     obj = hit4hmm2hspPR(**
#                     jdict
#                    )

0 of -1
500000 of -1
1000000 of -1
1500000 of -1
2000000 of -1
2500000 of -1
3000000 of -1
3500000 of -1
4000000 of -1

[SUMMARY]:
	[Task]: finshed  from  
	[Failrate]0 instances of 4440631 failed, ( 0.00% )
	[Duration]:Ended after 26.5034 sec
3777843
CPU times: user 27.8 s, sys: 1 s, total: 28.8 s
Wall time: 27.9 s


In [64]:
%load_ext line_profiler
def test():
    it = itertools.groupby(vals,key = lambda x:x['target_id'])
    it = itertools.chain(*[itertools.combinations(y,2) for x,y in it])
#     obj = next(it)
#     obj = hit4hmm2hspPR(**
#                                 pair__hit4hmm2hspDict(obj)
#                                )
#     return obj
    MySQL_hitPR( it,bsize = 1E5)
# %lprun -f MySQL_hitPR test()
%lprun -f hit4hmm2hspPR.__init__ test()

# %lprun -f pair__hit4hmm2hspDict test()
# te

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
inserting hit4hmm2hspPR for CATH nodes 0 of -1


[SUMMARY]:inserting hit4hmm2hspPR for CATH nodes 
	[Task]: finshed  from  
	[Failrate]0 instances of 99999 failed, ( 0.00% )
	[Duration]:Ended after 4.6798 sec


In [74]:
def MySQL_hitPR( it, bsize = 1E4):
    def callback(buf):
        hit4hmm2hspPR.objects.bulk_create(buf)
    
    reset_database_connection()
    behave = "inserting hit4hmm2hspPR for CATH nodes "
    
    try:
#         test
#     if 1:    
        with transaction.atomic():
            buf = []
            # for node in self.reverse_dict:
            c = counter([],INF=1, per = bsize, prefix = behave)
            for obj in it:
                obj = pair__hit4hmm2hspDict(obj)
#                 c.count()
                
                if obj:
                    obj = hit4hmm2hspPR(**
                        obj
                       )
                    buf.append(obj)
                    c.count()
                if not c.i % bsize:
        #             hit4cath2cath.objects.bulk_create(buf)
                    callback(buf)
                    buf = []    
                if not (c.i + 1)  % 1E5:
                    pass
#                     raise Exception()
            callback(buf)
    except Exception as e: 
        print str(e)
        pass
    c.summary()
    pass

In [80]:
%time lst = itertools.chain(*[itertools.combinations(y,2) for x,y in it])

CPU times: user 13.3 s, sys: 640 ms, total: 14 s
Wall time: 13.9 s


In [54]:
lst = list(next(it)[1])
i

In [67]:
def pair__hit4hmm2hspDict( lst ):
    hit1,hit2 = lst
    if hit1['node_id'] == hit2['node_id']:
        return None
    hits = lst
#     if hit1['id'] > hit2['id']:
#         hits=[hit2,hit1]
#     else:
#         hits=[hit1,hit2]
    geoavg  = ( 
            (hit2['end'] + 1 - hit2['start']) * 
            (hit1['end'] + 1 - hit1['start']) 
            ) **0.5
    overlap = (
    min( hit1['end'], hit2['end']) + 1 -
    max( hit1['start'],hit2['start'])
    )

    jdict = {
        'hit1_id':hits[0]['id'],
        'hit2_id':hits[1]['id'],
        'geoavg' : geoavg,
        'overlap': overlap,
    }
    return jdict

In [67]:
for hit1,hit2 in itertools.combinations( lst, 2):
    jdict = pair__hit4hmm2hspDict(hit1,hit2)
    obj = hit4hmm2hspPR(**jdict)
    buf.append( obj )
    break
# print obj.hit2

Query:HMM for 1.20.1050.10.56.0  
Target:14gsA01 from <class 'tst.models.seqDB'>


# Counting HMM-HMM intermediate sequence
## For Homsf nodes
## (sequence wise, Parallel)


In [25]:
matObj.dump()

[Msg] managed to dump D_raw to D_raw/CATH-S40-nr_H.
[Msg] managed to dump D_norm to D_norm/CATH-S40-nr_H.
[Msg] managed to dump D_both to D_both/CATH-S40-nr_H.
[Msg] managed to dump D_geoavg to D_geoavg/CATH-S40-nr_H.
[Msg] managed to dump hcounts to hcounts/CATH-S40-nr_H.


In [38]:
%%time
# from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')#[:1000]
matObj = ctmat( HMMprofile.objects, seqs = seqs, 
               alias = sDB.name , letter = 'H')

##### sDB is implicitly implied in "seqs"
# matObj.force = 1 
matObj.hit_sum(dump = 1)      #### summing hit-seq count for each hmmprofile
matObj.MySQL_hcount() #### put hit-seq count to MySQL database
matObj.Draw_para(pcount = 7, dump = 1 )
matObj.Dnorm(dump = 1)
matObj.Dgeoavg(dump = 1)
matObj.Dboth(dump = 1)
# matObj.

# matObj.dump('H_410')

[Msg] managed to dump hcounts to hcounts/CATH-S40-nr_H.

[SUMMARY]:inserting hit_summary for CATH nodes 
	[Task]: finshed  from  
	[Failrate]0 instances of 0 failed, ( 0.00% )
	[Duration]:Ended after 8.3181 sec
0
[Worker]0 of 21015
[Listener]0 of -1
1
[Worker]600 of 21015
[Listener]600 of -1
2
[Worker]1200 of 21015
[Listener]1200 of -1
3
[Worker]1800 of 21015
[Listener]1800 of -1
4
[Worker]2400 of 21015
[Listener]2400 of -1
5
[Worker]3000 of 21015
6
[Listener]3000 of -1
[Worker]3600 of 21015
[Listener]3600 of -1
7
[Worker]4200 of 21015
[Listener]4200 of -1
8
[Worker]4800 of 21015
[Listener]4800 of -1
9
[Worker]5400 of 21015
[Listener]5400 of -1
10
[Worker]6000 of 21015
[Listener]6000 of -1
11
[Worker]6600 of 21015
[Listener]6600 of -1
12
[Worker]7200 of 21015
[Listener]7200 of -1
13
[Worker]7800 of 21015
14
[Listener]7800 of -1
[Worker]8400 of 21015
[Listener]8400 of -1
15
[Worker]9000 of 21015
[Listener]9000 of -1
16
[Worker]9600 of 21015
17
[Listener]9600 of -1
[Worker]10200 of 21015

In [ ]:
%%time
# matObj.load()
print matObj.sDB
matObj.to_MySQL()


# Counting HMM-HMM intermediate sequence
## For S35 nodes
## (sequence wise, Parallel)


In [31]:
%%time
# from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')#[:1000]
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')

##### sDB is implicitly implied in "seqs"
# matObj.force = 1 
matObj.hit_sum()      #### summing hit-seq count for each hmmprofile
matObj.MySQL_hcount() #### put hit-seq count to MySQL database
matObj.Draw_para(pcount = 7)
matObj.Dnorm( )
matObj.dump()


inserting hit_summary for CATH nodes 0 of -1
inserting hit_summary for CATH nodes 10000 of -1
inserting hit_summary for CATH nodes 20000 of -1

[SUMMARY]:inserting hit_summary for CATH nodes 
	[Task]: finshed  from  
	[Failrate]0 instances of 21086 failed, ( 0.00% )
	[Duration]:Ended after 27.5227 sec
0
[Worker]0 of 21015
[Listener]0 of -1
1
[Worker]600 of 21015
[Listener]600 of -1
2
[Worker]1200 of 21015
[Listener]1200 of -1
3
[Worker]1800 of 21015
[Worker]2400 of 21015
[Listener]1800 of -1
4
[Listener]2400 of -1
5
[Worker]3000 of 21015
[Worker]3600 of 21015
[Listener]3000 of -1
6
[Listener]3600 of -1
7
[Worker]4200 of 21015
[Worker]4800 of 21015
[Listener]4200 of -1
8
[Listener]4800 of -1
9
[Worker]5400 of 21015
[Worker]6000 of 21015
[Listener]5400 of -1
10
[Worker]6600 of 21015
[Listener]6000 of -1
11
[Listener]6600 of -1
12
[Worker]7200 of 21015
[Listener]7200 of -1
13
[Worker]7800 of 21015
[Worker]8400 of 21015
[Listener]7800 of -1
14
[Listener]8400 of -1
15
[Worker]9000 of 21015


369000 of 1278380
370000 of 1278380
371000 of 1278380
372000 of 1278380
373000 of 1278380
374000 of 1278380
375000 of 1278380
376000 of 1278380
377000 of 1278380
378000 of 1278380
379000 of 1278380
380000 of 1278380
381000 of 1278380
382000 of 1278380
383000 of 1278380
384000 of 1278380
385000 of 1278380
386000 of 1278380
387000 of 1278380
388000 of 1278380
389000 of 1278380
390000 of 1278380
391000 of 1278380
392000 of 1278380
393000 of 1278380
394000 of 1278380
395000 of 1278380
396000 of 1278380
397000 of 1278380
398000 of 1278380
399000 of 1278380
400000 of 1278380
401000 of 1278380
402000 of 1278380
403000 of 1278380
404000 of 1278380
405000 of 1278380
406000 of 1278380
407000 of 1278380
408000 of 1278380
409000 of 1278380
410000 of 1278380
411000 of 1278380
412000 of 1278380
413000 of 1278380
414000 of 1278380
415000 of 1278380
416000 of 1278380
417000 of 1278380
418000 of 1278380
419000 of 1278380
420000 of 1278380
421000 of 1278380
422000 of 1278380
423000 of 1278380
424000 of 

930000 of 1278380
931000 of 1278380
932000 of 1278380
933000 of 1278380
934000 of 1278380
935000 of 1278380
936000 of 1278380
937000 of 1278380
938000 of 1278380
939000 of 1278380
940000 of 1278380
941000 of 1278380
942000 of 1278380
943000 of 1278380
944000 of 1278380
945000 of 1278380
946000 of 1278380
947000 of 1278380
948000 of 1278380
949000 of 1278380
950000 of 1278380
951000 of 1278380
952000 of 1278380
953000 of 1278380
954000 of 1278380
955000 of 1278380
956000 of 1278380
957000 of 1278380
958000 of 1278380
959000 of 1278380
960000 of 1278380
961000 of 1278380
962000 of 1278380
963000 of 1278380
964000 of 1278380
965000 of 1278380
966000 of 1278380
967000 of 1278380
968000 of 1278380
969000 of 1278380
970000 of 1278380
971000 of 1278380
972000 of 1278380
973000 of 1278380
974000 of 1278380
975000 of 1278380
976000 of 1278380
977000 of 1278380
978000 of 1278380
979000 of 1278380
980000 of 1278380
981000 of 1278380
982000 of 1278380
983000 of 1278380
984000 of 1278380
985000 of 

In [32]:
%%time
matObj.Dgeoavg(dump=1)
matObj.Dboth(dump=1)

0 of 1278380
1000 of 1278380
2000 of 1278380
3000 of 1278380
4000 of 1278380
5000 of 1278380
6000 of 1278380
7000 of 1278380
8000 of 1278380
9000 of 1278380
10000 of 1278380
11000 of 1278380
12000 of 1278380
13000 of 1278380
14000 of 1278380
15000 of 1278380
16000 of 1278380
17000 of 1278380
18000 of 1278380
19000 of 1278380
20000 of 1278380
21000 of 1278380
22000 of 1278380
23000 of 1278380
24000 of 1278380
25000 of 1278380
26000 of 1278380
27000 of 1278380
28000 of 1278380
29000 of 1278380
30000 of 1278380
31000 of 1278380
32000 of 1278380
33000 of 1278380
34000 of 1278380
35000 of 1278380
36000 of 1278380
37000 of 1278380
38000 of 1278380
39000 of 1278380
40000 of 1278380
41000 of 1278380
42000 of 1278380
43000 of 1278380
44000 of 1278380
45000 of 1278380
46000 of 1278380
47000 of 1278380
48000 of 1278380
49000 of 1278380
50000 of 1278380
51000 of 1278380
52000 of 1278380
53000 of 1278380
54000 of 1278380
55000 of 1278380
56000 of 1278380
57000 of 1278380
58000 of 1278380
59000 of 1

536000 of 1278380
537000 of 1278380
538000 of 1278380
539000 of 1278380
540000 of 1278380
541000 of 1278380
542000 of 1278380
543000 of 1278380
544000 of 1278380
545000 of 1278380
546000 of 1278380
547000 of 1278380
548000 of 1278380
549000 of 1278380
550000 of 1278380
551000 of 1278380
552000 of 1278380
553000 of 1278380
554000 of 1278380
555000 of 1278380
556000 of 1278380
557000 of 1278380
558000 of 1278380
559000 of 1278380
560000 of 1278380
561000 of 1278380
562000 of 1278380
563000 of 1278380
564000 of 1278380
565000 of 1278380
566000 of 1278380
567000 of 1278380
568000 of 1278380
569000 of 1278380
570000 of 1278380
571000 of 1278380
572000 of 1278380
573000 of 1278380
574000 of 1278380
575000 of 1278380
576000 of 1278380
577000 of 1278380
578000 of 1278380
579000 of 1278380
580000 of 1278380
581000 of 1278380
582000 of 1278380
583000 of 1278380
584000 of 1278380
585000 of 1278380
586000 of 1278380
587000 of 1278380
588000 of 1278380
589000 of 1278380
590000 of 1278380
591000 of 

1081000 of 1278380
1082000 of 1278380
1083000 of 1278380
1084000 of 1278380
1085000 of 1278380
1086000 of 1278380
1087000 of 1278380
1088000 of 1278380
1089000 of 1278380
1090000 of 1278380
1091000 of 1278380
1092000 of 1278380
1093000 of 1278380
1094000 of 1278380
1095000 of 1278380
1096000 of 1278380
1097000 of 1278380
1098000 of 1278380
1099000 of 1278380
1100000 of 1278380
1101000 of 1278380
1102000 of 1278380
1103000 of 1278380
1104000 of 1278380
1105000 of 1278380
1106000 of 1278380
1107000 of 1278380
1108000 of 1278380
1109000 of 1278380
1110000 of 1278380
1111000 of 1278380
1112000 of 1278380
1113000 of 1278380
1114000 of 1278380
1115000 of 1278380
1116000 of 1278380
1117000 of 1278380
1118000 of 1278380
1119000 of 1278380
1120000 of 1278380
1121000 of 1278380
1122000 of 1278380
1123000 of 1278380
1124000 of 1278380
1125000 of 1278380
1126000 of 1278380
1127000 of 1278380
1128000 of 1278380
1129000 of 1278380
1130000 of 1278380
1131000 of 1278380
1132000 of 1278380
1133000 of 1

In [35]:
%%time
print matObj.sDB
matObj.to_MySQL()

CATH-S40-nr_Ver:4_1_0
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 114 µs


# Testing Case that ignores all exceptions
## For S35 nodes
## (sequence wise, Parallel)


In [87]:
%%time
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')[:1000]
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S', alias = 'test')

##### sDB is implicitly implied in "seqs"
matObj.force = 1 
matObj.hit_sum()      #### summing hit-seq count for each hmmprofile
matObj.MySQL_hcount() #### put hit-seq count to MySQL database
matObj.Draw_para(pcount = 7)
matObj.Dnorm( )

matObj.dump()
# pk_dump( ('D_raw' , D_raw  ),  'test_' + matObj.letter + '_410')
# pk_dump( ('D_norm', D_norm ),  'test_' + matObj.letter + '_410')
# pk_dump( ('hcounts',matObj.hcounts), matObj.letter + '_410')

settings.USE_MODELLER
CPU times: user 9.34 s, sys: 352 ms, total: 9.69 s
Wall time: 39.5 s


#### Now calculating secondary statics (geometric average and doubly-zip all 3 matrix )

#### Now plug everything into MySQL

In [125]:
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'H')
matObj.load()
# matObj.D_both = wrap_mat( matObj.D_raw,matObj.D_norm)
matObj.to_MySQL()

settings.USE_MODELLER
[Msg] managed to load D_raw from D_raw/H_410
[Msg] managed to load D_norm from D_norm/H_410
[Msg] managed to load hcounts from hcounts/H_410
[Msg] failed  to load D_both from D_both/H_410
0 of -1

[SUMMARY]:
	[Task]: finshed zipping sparse matrix from  
	[Failrate]0 instances of 2931 failed, ( 0.00% )
	[Duration]:Ended after 0.1349 sec
inserting ISS hits between S35 0 of -1

[SUMMARY]:inserting ISS hits between S35 
	[Task]: finshed  from  
	[Failrate]0 instances of 2931 failed, ( 0.00% )
	[Duration]:Ended after 1.4490 sec


In [ ]:
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')
matObj.load()
# matObj.D_both = wrap_mat( matObj.D_raw,matObj.D_norm)
matObj.to_MySQL()

In [3]:
%%time
######################
## benchmark loading #
######################
seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'S')
matObj.load('S_410')

[Msg] managed to load D_raw from D_raw/S_410
[Msg] managed to load D_norm from D_norm/S_410
[Msg] managed to load D_both from D_both/S_410
[Msg] managed to load D_geoavg from D_geoavg/S_410
CPU times: user 42.2 s, sys: 676 ms, total: 42.8 s
Wall time: 42.9 s


In [15]:
%%time
###################
###################
###################
qset = hit4cath2cath.objects.filter(seqDB=sDB)
qset = list( qset )
# print qset.count()

CPU times: user 19.8 s, sys: 2.06 s, total: 21.9 s
Wall time: 30 s


In [19]:
qset = hit4hmm2hspPR.objects.annotate( stat = models.ExpressionWrapper( F('overlap')/F('geoavg'),output_field =models.FloatField() ) )
qset = qset.filter(stat__lte = 0.5)
qset.update(masked = 1)

In [27]:
qset = hit4hmm2hspPR.objects.filter(hit1__target__seqDB=sDB).exclude(masked=True)

In [29]:
matObj.field

'cath_node__id'

In [42]:
%%time
qset = hit4hmm2hspPR.objects.filter(hit1__target__seqDB=sDB).exclude(masked=True)
node1_field = 'hit1__query__%s' % matObj.field
node2_field = 'hit2__query__%s' % matObj.field
qset = qset.values_list( node1_field ,node2_field )
# qset = list(qset)

CPU times: user 56 ms, sys: 48 ms, total: 104 ms
Wall time: 111 ms


In [48]:
%%time
lst = []
print it[0]
for e in it:
    x,y = e
    lst += [sorted(
    [forward_dict[x],
    forward_dict[y]
    ]
    )]

(277741L, 280443L)
CPU times: user 8.81 s, sys: 600 ms, total: 9.41 s
Wall time: 9.17 s


In [62]:
%%time
lst = [
    tuple(sorted((forward_dict[x],
    forward_dict[y]
    )))
       for x,y in it]


CPU times: user 3.88 s, sys: 244 ms, total: 4.13 s
Wall time: 4.08 s


In [63]:
%%time
from collections import Counter
# count = Counter([set(x) for x in lst])
count = Counter(lst)
# count.update(lst)

CPU times: user 2.1 s, sys: 32 ms, total: 2.14 s
Wall time: 1.99 s


In [71]:
def mask_selection(self, **kwargs):
    qset = hit4hmm2hspPR.objects.filter(seqDB=self.sDB).annotate( stat = models.ExpressionWrapper( F('overlap')/F('geoavg'),output_field =models.FloatField() ) )
    if not kwargs:
        kwargs = {'stat__lte' : 0.5}
#     qset = qset.filter(stat__lte = 0.5)
    qset = qset.filter(**kwargs)
    return qset.update(masked = 1)


In [66]:
%%time
def Draw_hitPR(self, dump = 0):
    l = self.l
    OUTPUT = dok_matrix( (l + 1, l + 1), dtype = 'int')

    qset = hit4hmm2hspPR.objects.filter(hit1__target__seqDB=self.sDB).exclude(masked=True)
    node1_field = 'hit1__query__%s' % self.field
    node2_field = 'hit2__query__%s' % self.field
    it = list(qset.values_list( node1_field ,node2_field ))

    idxs = [
        tuple(sorted((
            self.forward_dict[x],
            self.forward_dict[y]
        )))
           for x,y in it]

    count = Counter(idxs)
    OUTPUT.update(count)
    self.D_raw = OUTPUT
    if dump:
        self.dump( name = self.name, attr = 'D_raw')
    return OUTPUT
D_raw = Draw_hitPR(matObj)
# print it[0]

CPU times: user 11.1 s, sys: 624 ms, total: 11.7 s
Wall time: 22.3 s


In [74]:
%%time
matObj.Dnorm()
matObj.Dgeoavg()
matObj.Dboth()
matObj.to_MySQL()

0 of 922591
1000 of 922591
2000 of 922591
3000 of 922591
4000 of 922591
5000 of 922591
6000 of 922591
7000 of 922591
8000 of 922591
9000 of 922591
10000 of 922591
11000 of 922591
12000 of 922591
13000 of 922591
14000 of 922591
15000 of 922591
16000 of 922591
17000 of 922591
18000 of 922591
19000 of 922591
20000 of 922591
21000 of 922591
22000 of 922591
23000 of 922591
24000 of 922591
25000 of 922591
26000 of 922591
27000 of 922591
28000 of 922591
29000 of 922591
30000 of 922591
31000 of 922591
32000 of 922591
33000 of 922591
34000 of 922591
35000 of 922591
36000 of 922591
37000 of 922591
38000 of 922591
39000 of 922591
40000 of 922591
41000 of 922591
42000 of 922591
43000 of 922591
44000 of 922591
45000 of 922591
46000 of 922591
47000 of 922591
48000 of 922591
49000 of 922591
50000 of 922591
51000 of 922591
52000 of 922591
53000 of 922591
54000 of 922591
55000 of 922591
56000 of 922591
57000 of 922591
58000 of 922591
59000 of 922591
60000 of 922591
61000 of 922591
62000 of 922591
63000

498000 of 922591
499000 of 922591
500000 of 922591
501000 of 922591
502000 of 922591
503000 of 922591
504000 of 922591
505000 of 922591
506000 of 922591
507000 of 922591
508000 of 922591
509000 of 922591
510000 of 922591
511000 of 922591
512000 of 922591
513000 of 922591
514000 of 922591
515000 of 922591
516000 of 922591
517000 of 922591
518000 of 922591
519000 of 922591
520000 of 922591
521000 of 922591
522000 of 922591
523000 of 922591
524000 of 922591
525000 of 922591
526000 of 922591
527000 of 922591
528000 of 922591
529000 of 922591
530000 of 922591
531000 of 922591
532000 of 922591
533000 of 922591
534000 of 922591
535000 of 922591
536000 of 922591
537000 of 922591
538000 of 922591
539000 of 922591
540000 of 922591
541000 of 922591
542000 of 922591
543000 of 922591
544000 of 922591
545000 of 922591
546000 of 922591
547000 of 922591
548000 of 922591
549000 of 922591
550000 of 922591
551000 of 922591
552000 of 922591
553000 of 922591
554000 of 922591
555000 of 922591
556000 of 9225

151000 of 922591
152000 of 922591
153000 of 922591
154000 of 922591
155000 of 922591
156000 of 922591
157000 of 922591
158000 of 922591
159000 of 922591
160000 of 922591
161000 of 922591
162000 of 922591
163000 of 922591
164000 of 922591
165000 of 922591
166000 of 922591
167000 of 922591
168000 of 922591
169000 of 922591
170000 of 922591
171000 of 922591
172000 of 922591
173000 of 922591
174000 of 922591
175000 of 922591
176000 of 922591
177000 of 922591
178000 of 922591
179000 of 922591
180000 of 922591
181000 of 922591
182000 of 922591
183000 of 922591
184000 of 922591
185000 of 922591
186000 of 922591
187000 of 922591
188000 of 922591
189000 of 922591
190000 of 922591
191000 of 922591
192000 of 922591
193000 of 922591
194000 of 922591
195000 of 922591
196000 of 922591
197000 of 922591
198000 of 922591
199000 of 922591
200000 of 922591
201000 of 922591
202000 of 922591
203000 of 922591
204000 of 922591
205000 of 922591
206000 of 922591
207000 of 922591
208000 of 922591
209000 of 9225

756000 of 922591
757000 of 922591
758000 of 922591
759000 of 922591
760000 of 922591
761000 of 922591
762000 of 922591
763000 of 922591
764000 of 922591
765000 of 922591
766000 of 922591
767000 of 922591
768000 of 922591
769000 of 922591
770000 of 922591
771000 of 922591
772000 of 922591
773000 of 922591
774000 of 922591
775000 of 922591
776000 of 922591
777000 of 922591
778000 of 922591
779000 of 922591
780000 of 922591
781000 of 922591
782000 of 922591
783000 of 922591
784000 of 922591
785000 of 922591
786000 of 922591
787000 of 922591
788000 of 922591
789000 of 922591
790000 of 922591
791000 of 922591
792000 of 922591
793000 of 922591
794000 of 922591
795000 of 922591
796000 of 922591
797000 of 922591
798000 of 922591
799000 of 922591
800000 of 922591
801000 of 922591
802000 of 922591
803000 of 922591
804000 of 922591
805000 of 922591
806000 of 922591
807000 of 922591
808000 of 922591
809000 of 922591
810000 of 922591
811000 of 922591
812000 of 922591
813000 of 922591
814000 of 9225

inserting ISS hits between S35 620000 of -1
inserting ISS hits between S35 630000 of -1
inserting ISS hits between S35 640000 of -1
inserting ISS hits between S35 650000 of -1
inserting ISS hits between S35 660000 of -1
inserting ISS hits between S35 670000 of -1
inserting ISS hits between S35 680000 of -1
inserting ISS hits between S35 690000 of -1
inserting ISS hits between S35 700000 of -1
inserting ISS hits between S35 710000 of -1
inserting ISS hits between S35 720000 of -1
inserting ISS hits between S35 730000 of -1
inserting ISS hits between S35 740000 of -1
inserting ISS hits between S35 750000 of -1
inserting ISS hits between S35 760000 of -1
inserting ISS hits between S35 770000 of -1
inserting ISS hits between S35 780000 of -1
inserting ISS hits between S35 790000 of -1
inserting ISS hits between S35 800000 of -1
inserting ISS hits between S35 810000 of -1
inserting ISS hits between S35 820000 of -1
inserting ISS hits between S35 830000 of -1
inserting ISS hits between S35 8

In [77]:
%%time
#################################################
### Do not call this method more than once! #####
#################################################
#################################################
matObj.to_MySQL()

inserting ISS hits between S35 0 of -1
inserting ISS hits between S35 10000 of -1
inserting ISS hits between S35 20000 of -1
inserting ISS hits between S35 30000 of -1
inserting ISS hits between S35 40000 of -1
inserting ISS hits between S35 50000 of -1
inserting ISS hits between S35 60000 of -1
inserting ISS hits between S35 70000 of -1
inserting ISS hits between S35 80000 of -1
inserting ISS hits between S35 90000 of -1
inserting ISS hits between S35 100000 of -1
inserting ISS hits between S35 110000 of -1
inserting ISS hits between S35 120000 of -1
inserting ISS hits between S35 130000 of -1
inserting ISS hits between S35 140000 of -1
inserting ISS hits between S35 150000 of -1
inserting ISS hits between S35 160000 of -1
inserting ISS hits between S35 170000 of -1
inserting ISS hits between S35 180000 of -1
inserting ISS hits between S35 190000 of -1
inserting ISS hits between S35 200000 of -1
inserting ISS hits between S35 210000 of -1
inserting ISS hits between S35 220000 of -1
in

In [ ]:
sDB.hit4cath2cath

In [76]:
%%time
qset  = hit4cath2cath.objects.filter(seqDB = matObj.sDB)
qset.delete()

CPU times: user 24 ms, sys: 12 ms, total: 36 ms
Wall time: 19min 2s


In [26]:
%%time
qset = list(qset)

CPU times: user 1min 3s, sys: 5.59 s, total: 1min 9s
Wall time: 1min 17s


In [25]:
print qset.count()

3777843


In [16]:
%load_ext line_profiler
#####################################################
##### identifying bottleneck in MySQL insertion #####
#####################################################

def test():
    for i in range(10):
        hitlist_compare(sDB=sDB)

# test()
# %lprun -f hit4cath2cath.__init__ test()
%lprun -f hitlist_compare test()

In [ ]:
%load_ext line_profiler
#####################################################
##### identifying bottleneck in MySQL insertion #####
#####################################################

def test():
#     matObj.to_MySQL(bsize = 1E4, ignoreFunc = lambda v: v < 0.0, dry_run = True, cstop = 2000)
    matObj.to_MySQL(bsize = 1E4, ignoreFunc = lambda v: v < 0.0, dry_run = True, )
    
# test()
# %lprun -f hit4cath2cath.__init__ test()
%lprun -f matObj.to_MySQL test()

In [85]:
%load_ext line_profiler
reset_database_connection()
sDB = seqDB.objects.get(name = 'CATH-S40-nr')
def test():
    for i in range(10):
        hitlist_compare(sDB = sDB)
%lprun -f hitlist_compare test()



The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [23]:
%load_ext line_profiler
%reload_ext line_profiler
reset_database_connection()
sDB = seqDB.objects.get(name = 'CATH-S40-nr')
def test():
    for i in range(3000):
        draft_hitPR(matObj)
%lprun -f draft_hitPR test()



The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
qset = classification.objects.filter(level__letter = 'H').distinct()   
rv_field = reverse_field[ 'H' ]
sDB_field  = rv_field + "__seqDB"
text_field = rv_field.replace('__hits','__text')
rvid_field = rv_field.replace('__hits','__hit4hmm2hsp__id')
fs = ['id',rv_field, rvid_field, sDB_field]

In [34]:
%%time
reset_database_connection()
seqs = sDB.sequence_set.all().defer('hmmprofile__text')#[:1000]
matObj = ctmat( HMMprofile.objects, seqs = seqs, 
               alias = sDB.name , letter = 'H')

CPU times: user 40 ms, sys: 8 ms, total: 48 ms
Wall time: 686 ms


In [35]:
d,s = cache_hitlist(matObj, dump = 1)

[Msg] managed to dump cache_hitlist to cache_hitlist/CATH-S40-nr_H.
[Msg] managed to dump cache_hitids to cache_hitids/CATH-S40-nr_H.


In [21]:
# matObj.load(attr='D_raw')
ISS_count = sum( v for i,j,v in  using_tocoo_izip(matObj.D_raw))
print ISS_count

51690


In [60]:
from itertools import groupby
def cache_hitdict(self,dump = 0):
    reset_database_connection()   
#     if not self.
    if not self.cache_hitids:
        lst = list(self.hmms.values_list('hit4hmm2hsp__id',flat = True).distinct())
        self.cache_hitids = [x for x in lst if x]
        if dump:
            self.dump( name = self.name, attr = 'cache_hitids')
            
    N = 100000  ### give batch MySQL packet around 4000kbytes
    N = 1000000 
    batches = batch_qs(self.cache_hitids, N)
    def mapF(ids):
#         hitdict = hit4hmm2hsp.objects.defer("query").in_bulk(ids)
        hitdict = list(hit4hmm2hsp.objects.filter(id__in=ids).values())
        
#         hitdict = hit4hmm2hsp.objects.defer("query__text").in_bulk(ids)
#         hitdict = hit4hmm2hsp.objects.in_bulk(ids)
        
        return hitdict
    # map(batches, mapF)
    # hitD = map(mapF,batches)
    # %time hitD = mapF(next(batches))
    OUTPUT = {}
    c = counter(range( len(self.cache_hitids)  // N + 1 ), per = 1)
    for batch in batches:
        r = mapF(batch)
        return r       
        OUTPUT.update( r )
        c.count()
#         raise Exception()
#         break
    self.cache_hitdict = OUTPUT
    if dump:
            self.dump( name = self.name, attr = 'cache_hitdict')
    return OUTPUT

#     return lst
def cache_hitlist(self, dump = 0):
    rv_field = self.rfield
    sDB_field  = self.rfield + "__seqDB"
    text_field = self.rfield.replace('__hits','__text')
    rvid_field = self.rfield.replace('__hits','__hit4hmm2hsp__id')   
    self.fs = ['id',rv_field, rvid_field, sDB_field]
    self.tfield = text_field
    qset = classification.objects.filter(id__in=self.reverse_dict)
#     qset = qset[:10000]
    hitlist = qset.values_list(*self.fs)    
#     hitlist = qset.values_list(*self.fs).distinct()
    # hitlist = qset.values(*fs).distinct() 
    hitlist = list(hitlist)
    s = list(set(x[2] for x in hitlist))
    self.cache_hitids =  s
    
    keyF = lambda x: x[0]
    hitlist = sorted(hitlist,key = keyF)
#     return hitlist
    it = groupby(hitlist, key = keyF) 
#     return (it)
    d = {}
    for k,g in it:
        lst = [e[1:] for e in g if e[1]]
        val = {seqid:hitid for seqid,hitid,sid in lst if sid == self.sDB_id}
        d[k] = val
    self.cache_hitlist = d
    if dump:
        self.dump( name = self.name, attr = 'cache_hitlist')
        self.dump( name = self.name, attr = 'cache_hitids')
            
    return (d,s)


In [24]:
forward_field

{'A': 'cath_node__parent__parent__parent__id',
 'C': 'cath_node__parent__parent__parent__parent__id',
 'H': 'cath_node__parent__id',
 'S': 'cath_node__id',
 'T': 'cath_node__parent__parent__id'}

In [32]:
d = matObj.cache_hitdict.values()[0]
# d.query.text

u'HMMER3/f [3.1b1 | May 2013]\nNAME  cath|4_1_0|16pkA01/5-192\nLENG  174\nALPH  amino\nRF    yes\nMM    no\nCONS  yes\nCS    no\nMAP   yes\nDATE  Tue Apr 21 18:28:53 2015\nNSEQ  4531\nEFFN  3.685897\nCKSUM 2001751443\nSTATS LOCAL MSV      -10.2947  0.70718\nSTATS LOCAL VITERBI  -11.0146  0.70718\nSTATS LOCAL FORWARD   -4.7069  0.70718\nHMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   \n            m->m     m->i     m->d     i->m     i->i     d->m     d->d\n  COMPO   2.51301  4.45183  2.84451  2.60480  3.39087  2.89997  3.65633  2.89307  2.59569  2.40331  3.72733  2.99641  3.37375  3.12523  2.91088  2.64923  2.85482  2.60055  5.14637  3.67594\n          2.68618  4.42225  2.77519  2.73123  3.46354  2.40513  3.72494  3.29354  2.67741  2.69355  4.24690  2.90347  2.73739  3.18146  2.89801  2.37887  2.77519  2.98518  4.58477  3.61503\n          0.14683  3

In [27]:
matObj.seqs[:100][0].hit4hmm2hsp_set

In [ ]:
def batch_worker(seqset,field="cath_node", q = None, c0 = None, c1 = None ):
    count = Counter()
    for seq in seqset:
        c0.count()
        ids = seq.hit4hmm2hsp_set

        obj = hitP_obj(
        id = i,
#         acc = hitD[hit1_id].target.acc,
        hit1 = hitD[hit1_id],
        hit2 = hitD[hit2_id]
        ) 
        obj.fill()
        
#         ids = set( seq.hmmprofile_set.values_list( field,flat = True) )
    #             pairs = it.combinations(sorted(hmmids),2)
        pairs = itools.combinations( sorted(ids) ,2)
        count.update(pairs)
    q.put(count)
#     c1.count()
    return 


In [22]:
def draft_hitPR(self,node1__id = 309754, node2__id = 310524, cache = None, hitD = None):
    if not cache:
        cache = self.cache_hitlist
    hitD = hitD or self.cache_hitdict
# #     sDB_id = self.sDB_
#     hitlist1 = cache[node1__id]
#     hitlist2 = cache[node2__id]
# #     hits1_ids = {seqid:hitid for seqid,hitid,sid in hitlist1 if sid == self.sDB_id}
# #     hits2_ids = {seqid:hitid for seqid,hitid,sid in hitlist2 if sid == self.sDB_id}
#     hits1_ids = {seqid:hitid for seqid,hitid,sid in hitlist1 }
#     hits2_ids = {seqid:hitid for seqid,hitid,sid in hitlist2 }

    hits1_ids = cache[node1__id]
    hits2_ids = cache[node2__id]

    inter_ids = set(hits1_ids) & set(hits2_ids)
    
    hits1_ids = [hits1_ids[i] for i in inter_ids]
    hits2_ids = [hits2_ids[i] for i in inter_ids]
    hitD = hitD or hit4hmm2hsp.objects.in_bulk( hits1_ids + hits2_ids )

    qset = []
    for i,hit1_id,hit2_id in izip(
            inter_ids,hits1_ids,hits2_ids
            ):
        obj = hitP_obj(
        id = i,
#         acc = hitD[hit1_id].target.acc,
        hit1 = hitD[hit1_id],
        hit2 = hitD[hit2_id]
        ) 
        obj.fill()
    
        qset += [ obj ]
    pass
def threshold_hitPR(self):
    
    pass

In [105]:
%%time
reset_database_connection()
# hitlist = qset.values_list(*fs).filter(**{sDB_field:sDB}).distinct() 
hitlist = qset.values_list(*fs).distinct() 
hitlist = list(hitlist)
# qset.

# qset.count()
# print qset 
# qset [1]

CPU times: user 9.68 s, sys: 724 ms, total: 10.4 s
Wall time: 57 s


In [106]:
%%time
reset_database_connection()
hitids =  set(x[2] for x in hitlist)

# hitinfos = list(hit4hmm2hsp.objects.filter(id__in=hitids))
# hitdict = hit4hmm2hsp.objects.in_bulk( )

CPU times: user 1.24 s, sys: 316 ms, total: 1.56 s
Wall time: 1.51 s


In [ ]:
batch_qs(list(hitids))

In [ ]:
%%time
reset_database_connection()
N = 10000
# N = len(hitids) + 1
ids = list(hitids)[:N]
hitdict = hit4hmm2hsp.objects.in_bulk(ids)
print hitdict.__sizeof__() / float(N)
print hitdict.__sizeof__() / float(N) * len(hitids) 

/home/shouldsee/.local/lib/python2.7/site-packages/django/db/backends/mysql/base.py:101: Warning: Memory capacity of 8388608 bytes for 'range_optimizer_max_mem_size' exceeded. Range optimization was not done for this query.
  return self.cursor.execute(query, args)


In [ ]:
print 'hi'

In [ ]:
# keyF = lambda x: x['id']
keyF = lambda x: x[0]
hitlist = sorted(hitlist,key = keyF)

In [163]:
# %%timeit
from itertools import groupby
it = groupby(hitlist, key = keyF) 
d = {}
for k,g in it:
    d[k] =[e[1:] for e in g if e[1]]

In [151]:
# %%timeit
from itertools import groupby
it = groupby(hitlist, key = keyF) 
d = {}
for k,g in it:
#     d[k] = [x for x in g]
    d[k] = list(g)  

In [166]:
%load_ext line_profiler
reset_database_connection()
sDB = seqDB.objects.get(name = 'CATH-S40-nr')
def test():
    for i in range(10):
        hitlist_compare(sDB = sDB,cache = d)
%lprun -f hitlist_compare test()

# hitlist_compare(sDB = sDB)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [32]:
# reverse_field
forward_field

{'A': 'cath_node__parent__parent__parent__id',
 'C': 'cath_node__parent__parent__parent__parent__id',
 'H': 'cath_node__parent__id',
 'S': 'cath_node__id',
 'T': 'cath_node__parent__parent__id'}

In [165]:
def hitlist_compare(node1__id = 309754, node2__id = 310524, sDB = None, cache = None, **kwargs):
    sDB_id = sDB.id
    
    if cache:
        hitlist1 = d[node1__id]
        hitlist2 = d[node2__id]        
    else:

        qset1 = classification.objects.filter(id = node1__id)
        qset2 = classification.objects.filter(id = node2__id)

        node1 = qset1[0]
        node2 = qset2[0]


        rv_field = reverse_field[ node1.level.letter ]
        sDB_field  = rv_field + "__seqDB"
        text_field = rv_field.replace('__hits','__text')
        rvid_field = rv_field.replace('__hits','__hit4hmm2hsp__id')

        fs = [rv_field, rvid_field, sDB_field]
        hitlist1 = qset1.values_list(*fs).distinct() 


        rv_field = reverse_field[ node2.level.letter ]
        sDB_field  = rv_field + "__seqDB"
        text_field = rv_field.replace('__hits','__text')
        rvid_field = rv_field.replace('__hits','__hit4hmm2hsp__id')

        fs = [rv_field, rvid_field, sDB_field]
    #     hitlist2 = qset2.values_list(*fs).distinct() 
        hitlist2 = qset2.values_list(*fs)

    #     hitlist2 = list(hitlist2)


    #     hitlist1 = [e[1:] for e in hitlist1]
    #     hitlist2 = [e[1:] for e in hitlist2]

    
    hits1_ids = {seqid:hitid for seqid,hitid,sid in hitlist1 if sid == sDB_id}
    hits2_ids = {seqid:hitid for seqid,hitid,sid in hitlist2 if sid == sDB_id}
    inter_ids = set(hits1_ids) & set(hits2_ids)

    hits1_obj = hit4hmm2hsp.objects.in_bulk( [hits1_ids[i] for i in inter_ids] )
    hits2_obj = hit4hmm2hsp.objects.in_bulk( [hits2_ids[i] for i in inter_ids] )

    # print len(inter_ids)

    # qset = [ 
    # hitP_obj(
    #     id = i,
    #     acc = hit1.target.acc,
    #     hit1 = hit1,
    #     hit2 = hit2) for i,hit1,hit2 in 
    #     izip(
    #         inter_ids,hits1_obj.values(),hits2_obj.values()
    #     )

    # ]

    qset = []

    for i,hit1,hit2 in izip(
            inter_ids,hits1_obj.values(),hits2_obj.values()
            ):
        obj = hitP_obj(
        id = i,
        acc = hit1.target.acc,
        hit1 = hit1,
        hit2 = hit2) 
        obj.fill()
    
        qset += [ obj ]


    return qset


In [39]:
# def pull_hitPR()

def Draw_para(self,  bsize = 600, pcount = 5, dump = 0 ):
    class MyManager(SyncManager): pass
    MyManager.register('counter',counter)

    field = self.field
    self.seqs = self.seqs.prefetch_related('hmmprofile_set__' + self.field.rstrip('__id') ) 

    ##### Actually firing subprocesses #####

    reset_database_connection()
    INPUT = self.seqs
    INPUTs = batch_qs( INPUT, bsize )
    l = self.l
    OUTPUT = dok_matrix( (l + 1, l + 1), dtype = 'int')

    local_listener = listener      
    local_worker = batch_worker
    # print __name__
    # if __name__=='__main__':
    if 1:
            global m
            m = MyManager()
            m.start()

            c0 = m.counter( range(INPUT.count()),INF=0, ifprint = 1,  prefix = '[Worker]',
                          per = bsize,)
            c1 = m.counter( [],INF=1, ifprint = 1, prefix = '[Listener]',
                          step = bsize, per = bsize, )
            fdict = m.dict(self.forward_dict)
            q = m.Queue();          

            if 1:
                pool = mp.Pool( pcount )
                watcher  = mp.Process( target = local_listener, args = [OUTPUT], kwargs = {'q':q,'c0':c0,'c1':c1,
                                                                          'fdict':fdict
                                                                                          },)
                watcher.start()
                jobs = []

                for INPUT_curr in INPUTs:
            #         job = pool.Process
                    job  = pool.apply_async(local_worker, (INPUT_curr,), {'q':q, 'field': self.field,'c0':c0,'c1':c1,
                                                                         })
                    jobs.append(job)
                for job in jobs:
                    job.get()

                q.put(None)
                watcher.join()
                OUTPUT = q.get() 
            pool.close()
            pool.join()
            if not self.force:
                test__raw( OUTPUT, reverse_dict = self.reverse_dict, letter = self.letter, seqDB_curr = self.sDB)
    # D_raw = OUTPUT
    self.D_raw = OUTPUT        
    if dump:
        self.dump( name = self.name, attr = 'D_raw')


    return OUTPUT